## combining chromosome SNP data per sample into WES SNP data
### also add EB data if available to filter out bad SNPs

In [ ]:
# get the code
import sys
sys.path.append('../code')
from combineCNVdata import get_covNsnp

In [ ]:
# HOME
home = '/Users/mahtin'
home = '/Users/martinscience'
# standard paths
testdata = os.path.join(home,"Dropbox/Icke/Work/somVar/testdata")
static_path = os.path.join(home, "Dropbox/Icke/Work/static")
cluster_path = os.path.join(home, "/Users/martinscience/mount")
tooldata = os.path.join(home, "Dropbox/Icke/Work/somVar/tooldata")

# tool-specific paths
shell_path = "../shell"
# 
cnvdata = os.path.join(tooldata, "myCNVdata")
output_path = os.path.join(cnvdata, "output")

# the path to the input data
cnv_path = os.path.join(cnvdata, "cnv")
cnvPON_path = cnv_path = os.path.join(cnvdata, "chromCov")
cnv_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/cnv")
cnvPON_path = os.path.join(cluster_path, "scratch/develop/PONcoverage/chromCov")

In [ ]:
cnvPON_path

In [ ]:
sample = "01_A"
snp_df, cov_df = get_covNsnp(sample, sample_cnv_path=cnv_path, PON_cnv_path=cnvPON_path, verbose=False)

In [ ]:
snp_df[:10]

In [ ]:
cov_df[:10]

## rolling window metrices

In [ ]:
def rolling_SNP_metrix(snp_df, chrom='',window_size=20, center=0.5):
    '''
    creates metrices to define heteroSNP gaps
    '''
    
    # reduce to hetero_SNPs
    df = snp_df.query('Chr == @chrom')
    # compute the heteroSNP center as the mean of all VAFs

    df.loc[:,'offCenter'] = np.abs(df['VAF'] - center)
    df.loc[:,'stdL'] = df.rolling(window_size)['VAF'].std()
    df.loc[:,'stdR'] = df.shift(-window_size + 1)['stdL']
    df.loc[:,'sumL'] = df.rolling(window_size)['offCenter'].sum()
    df.loc[:,'sumR'] = df.shift(-window_size + 1)['sumL']
    df.loc[:,'dVAFL'] = df.rolling(window_size)['deltaVAF'].sum()
    df.loc[:,'dVAFR'] = df.shift(-window_size + 1)['dVAFL']
    # fill up the margins
    for col in ['stdL', 'sumL', 'dVAFL']:
        df[col] = df[col].fillna(method='bfill')
    for col in ['stdR', 'sumR', 'dVAFR']:
        df[col] = df[col].fillna(method='ffill')
    df.loc[:,'scoreL'] = df['sumL'] + df['dVAFL']
    df.loc[:,'scoreR'] = df['sumR'] + df['dVAFL']
    return df

In [ ]:
def mark_heteroSNP(snp_df, config={}):
    '''
    creates the heteroSNP metrices for the entire exome
    '''
    # reduce the snp_df
    VAFmin = config['VAFlimits'][0]
    VAFmax = config['VAFlimits'][1]
    minDepth = config['minDepth']
    minEBscore = config['minEBscore']
    df = snp_df.query('@VAFmin < VAF < @VAFmax').query('Depth >= @minDepth and EBscore > @minEBscore')
    # find the center
    center = df['VAF'].mean()
    print("heteroSNP centered around", center)
    # get the local VAF difference
    df['deltaVAF'] = np.abs(df['VAF'] - df.shift(1)['VAF']).fillna(0)
    
    # get the 
    std_dfs = []
    for chrom in df['Chr'].unique():
        std_df = rolling_SNP_metrix(df, chrom=chrom, window_size=config['windowSize'], center=center)
        std_dfs.append(std_df)
    std_df = pd.concat(std_dfs)
    
    # normalize the values globally
    std_df.loc[:,'scoreL'] = std_df['scoreL'] / std_df['scoreL'].max()
    std_df.loc[:,'scoreR'] = std_df['scoreR'] / std_df['scoreR'].max()
    # maximum of scoreDiff marks the breakpoints
    std_df.loc[:,'scoreDiff'] = std_df['scoreR'] - std_df['scoreL']
    std_df.loc[:,'scoreDiff'] = (std_df['scoreDiff'] / std_df['scoreDiff'].max() / 2) + center
    return std_df

In [ ]:
config = {
    'VAFlimits':[0.05,0.95],
    'windowSize': 50,
    'minEBscore':0.5,
    'minDepth': 30
    
}
std_df = mark_heteroSNP(snp_df, config=config)
std_df

### find the CNV regions

### expand the regions according to leftCNV and rightCNV

In [ ]:
df = std_df.query('Chr == "chr17"')

In [ ]:
df['cnvL'] = (df['scoreL'] > 0.4).astype(int)
df['cnvR'] = (df['scoreR'] > 0.4).astype(int)
df[40:90]

### now do a rolling scan for more than x zeros and fill that up 
#### increases the 

## OUTPUT

In [ ]:
std_df.to_csv(os.path.join(output_path, 'heteroSNP/01_A.snp'), sep='\t', index=False)
cov_df.to_csv(os.path.join(output_path, 'covDif/01_A.snp'), sep='\t', index=False)

In [ ]:
snp_df

### combine SNP with local coverage info

In [ ]:
std_df